In [1]:
!pip install pytesseract
!pip install pytube
# Install Tesseract OCR in Colab
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
# Install yt-dlp
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 984.9 kB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (3,178 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121753 

In [2]:
!pip install openai

import os
from openai import OpenAI
from google.colab import userdata

def get_completion(prompt, model="gpt-4-0125-preview"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    client = OpenAI(
        # This is the default and can be omitted
        api_key=userdata.get('OPENAI_API_KEY'),
    )
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00


In [3]:
import cv2
import pytesseract
from pytube import YouTube
from PIL import Image
from difflib import SequenceMatcher
import os
import subprocess

def clean_text(text):
    # Remove extra spaces
    text = ' '.join(text.split())

    # Remove blank lines
    lines = text.split('\n')
    non_blank_lines = [line for line in lines if line.strip()]
    cleaned_text = '\n'.join(non_blank_lines)

    return cleaned_text


def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()


# def download_yt_video(video_url):
#   yt = YouTube(video_url)
#   video = yt.streams.filter(file_extension='mp4').first()
#   video.download('.')

#   # Update Tesseract OCR path for Colab
#   pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

#   # Path to the downloaded video file
#   video_path = './' + video.default_filename
#   return video_path

def download_yt_video(video_url):
    # Path to the downloaded video file
    video_path = './' + video_url[-11:] + '.mp4'
    print(video_path)

    # Use yt-dlp to download the best quality mp4 video
    command = f'yt-dlp -f "best[ext=mp4]" "{video_url}" -o "{video_path}"'
    subprocess.run(command, shell=True, check=True)

    return video_path


def run_ocr_on_video(video_path, read_rate=250):
  # Open the video file
  video = cv2.VideoCapture(video_path)

  # Initialize an empty string to store the extracted text
  extracted_text = []

  # Loop through the frames of the video
  counter = 0
  previous_text = "*****"
  while video.isOpened():
    counter += 1
    # Read every "read_rate" frame
    for i in range(read_rate):
      ret, frame = video.read()

    # Check if frame reading was successful
    if not ret:
        break

    # Convert the frame to PIL Image format
    image = Image.fromarray(frame)

    # Perform OCR on the image
    text = pytesseract.image_to_string(image)
    text = clean_text(text)
    if text and text != previous_text:
      if text.find(previous_text) < 0 and similarity(text, previous_text) < 0.5:
        extracted_text += [text]
      else:
        extracted_text[-1] = text
      print(counter, ": ", len(extracted_text), ": ", text, "")
      previous_text = text

  # Release the video capture
  video.release()
  return extracted_text


In [4]:
def yt_video_to_text(video_url):
  video_path = download_yt_video(video_url)
  extracted_text = run_ocr_on_video(video_path)
  return extracted_text


def text_to_bullets(extracted_text, prompt_body):
  prompt = f"""{prompt_body} ```{"| ".join(extracted_text)}```
  """
  print(prompt)
  response = get_completion(prompt)
  return response


def yt_video_to_bullet(video_url, prompt_body):
  extracted_text = yt_video_to_text(video_url)

  response = text_to_bullets(extracted_text, prompt_body)
  return response


In [ ]:
# YouTube video URL

video_url = 'https://www.youtube.com/watch?v=hDRDx5HtiBs'
prompt_body = """
The text is extracted from slides images and each slide is
 separated by a |. Your task is to
 extract all the main points in the text in organized bullet and
 sub-bullet points. Don't miss any points in the text, don't change order of the text, don't change the words.

Text:
"""

response = yt_video_to_bullet(video_url, prompt_body)
print(response)


./hDRDx5HtiBs.mp4
6 :  1 :  i J Pr , AL ls 
7 :  2 :  Sq = Caroline Uhler Director, Eric & Wendy Schmidt Center 
8 :  2 :  Caroline Uhler Director, Eric & Wendy Schmidt Center <4 re 
9 :  2 :  P< Caroline Uhler S Director, Eric & Wendy Schmidt Center 
11 :  3 :  . 
17 :  4 :  ERIC AND WENDY TEI Guiding Diffusion Models Towards Generative Optimization Mengdi Wang Princeton Al2 Initiative Electrical and Computer Engineering, Center for Statistics & Machine Learning Princeton University 
18 :  4 :  sien ERIC AND WENDY CHMIDT CENTER AT BROAD INSTITUTE Guiding Diffusion Models Towards Generative Optimization Mengdi Wang Princeton Al2 Initiative Electrical and Computer Engineering, Center for Statistics & Machine Learning Princeton University 
21 :  4 :  ERIC AND WENDY SCHMIDT CENTER Guiding Diffusion Models Towards 3ROAD Generative Optimization Ley Mengdi Wang Princeton Al2 Initiative Electrical and Computer Engineering, Center for Statistics & Machine Learning Princeton University 
22 :  5

In [ ]:
prompt_body = """
The text is extracted from slides images and each slide is
 separated by a |. Your task is to
 extract all the papers referenced in these slides. Don't miss any points in the text, don't change order of the text, don't change the words.

Text:
"""

response = yt_video_to_bullet(video_url, prompt_body)
print(response)

In [ ]:
# # YouTube video URL with long text (Conference)
# video_url = 'https://www.youtube.com/watch?v=hE135guhTQo'

# prompt_body = """
# The text is extracted from slides images and each slide is
#  separated by a |. Your task is to extract all the main points in the following
#  text in organized bullet points.
#  Don't miss any points in the text, don't change order of the text,
#  don't change the words.

# Text:
# """

# extracted_text = yt_video_to_text(video_url)

# response1 = text_to_bullets(extracted_text[:35], prompt_body)
# response2 = text_to_bullets(extracted_text[35:], prompt_body)
# prompt = f"""
# Please merge these two bullet point texts.
# Don't miss any points in the text, don't change order of the text,
#  don't change the words.

# Summary1: ```{response1}```
# Summary2: ```{response2}```
# """

# final_response = get_completion(prompt)
# print(final_response)